In [1]:
import pandas as pd
import numpy as np

In [2]:
routeExcelFileEstadosFinancieros="20250317_estados_financieros_ahorro_credito_enero25 (1).xlsx"

def read_data(routeExcelFile):
    data=pd.read_excel(routeExcelFile,skiprows=3)
    return data

def cleaningRows(data):
    data.drop(0, axis=0, inplace=True)
    data.index=data["CUENTA"].astype(int)
    data.drop("CUENTA",axis=1,inplace=True)
    return data

In [3]:
data= read_data(routeExcelFileEstadosFinancieros)
data= cleaningRows(data)
data.head()

,NOMBRE CUENTA,COOPERATIVA DE EMPLEADOS DE CAFAM,COOPERATIVA DE TRABAJADORES DE LA INDUSTRIA MILITAR,COOPERATIVA DE LOS PROFESIONALES DE LA SALUD COASMEDAS,COOPERATIVA DE AHORRO Y CREDITO PARA EL BIENESTAR SOCIAL,COOPERATIVA PARA EL BIENESTAR SOCIAL,COOPERATIVA FINANCIERA SAN FRANCISCO,COOPERATIVA MULTIACTIVA DE LA AVIACION CIVIL COLOMBIANA,COOPERATIVA DE EMPLEADOS DE DOW COLOMBIA,PROGRESSA ENTIDAD COOPERATIVA DE AHORRO Y CR??DITO,...,COOPERATIVA DE AHORRO Y CREDITO UNION COLOMBIANA,COOPERATIVA DE AHORRO Y CREDITO DE DROGUISTAS DETALLISTAS,COOPERATIVA DE AHORRO Y CREDITO COLANTA,MICROEMPRESAS DE COLOMBIA COOPERATIVA DE AHORRO Y CREDITO,COOPERATIVA DE AHORRO Y CREDITO CAJA UNION COOPERATIVA,COOPERATIVA ESPECIALIZADA DE AHORRO Y CREDITO AFROAMERICANA,COPERATIVA ESPECIALIZADA DE AHORRO Y CREDITO CANAPRO,LA COOPERATIVA DE AHORRO Y CREDITO SUCREDITO,COOPERTAIVA ESPECIALIZADA DE AHORRO Y CREDITO TAX LA FERIA,COOPERATIVA SUYA
CUENTA,,,,,,,,,,,,,,,,,,,,,
100000,ACTIVO,157304556861.059998,12531395186.16,389688917238.52002,172908896258.220001,133136789455.009995,12778969760.629999,76274160451.820007,40557107161.510002,227985516240.339996,...,46926017900.639999,302437201540.349976,377180050400,224408178381.75,11508610698.709999,8951768603.32,97391732713.389999,122592478140.649994,45276385837.519997,93426219948.589996
110000,EFECTIVO Y EQUIVALENTE AL EFECTIVO,25431120091.290001,1424903504.06,50132055234.370003,14486271584.690001,2269850000.29,3615899368.22,9827843953.379999,3383000838.93,11084216353.92,...,9295855661.68,46327685695.699997,50450645932,35817320750.029999,1465065822.03,1483652333.68,10478556830.1,8018608094.47,12452855387.24,16881798543.02
110500,CAJA,78129332.73,26810209.4,950282723.7,1000000,315662902,262125271.93,187929350,780000,153823490,...,2500000,2000000,7976775969,2402359711.93,25865303.02,883006467.3,365742637.34,88052564.49,64579723.46,2996191900.96
110505,CAJA GENERAL,67680842.73,23386709.4,945432723.7,0,309968902,262125271.93,174406350,0,152823490,...,0,0,7973225969,2392009711.93,25839252.8,882406467.3,361742637.34,83052564.49,62629723.46,2988362900.96
110510,CAJA MENOR,10448490,3423500,4850000,1000000,5694000,0,13523000,780000,1000000,...,2500000,2000000,3550000,10350000,26050.22,600000,4000000,5000000,1950000,7829000


In [ ]:
dataModelCamel="Ejercicio modelo CAMEL (2).xlsx"

def readExcelModelCamel(routeExcelFileModelCamel):
    data=pd.read_excel(routeExcelFileModelCamel)
    data.drop(["Unnamed: 0","%Participacón del Indicador","Resultado del indicador","Calificación"],axis=1,inplace=True)
    data = data.dropna(how='all')
    return data[:-1]

def fillCols(execFile, cols: list):
    execFile[cols] = execFile[cols].ffill()
    return execFile



#calculo indicador capital(C)
def quebrantoPatrimonial(excelFile,nombreEmpresa):
    result=excelFile.at[300000,nombreEmpresa]/excelFile.at[310000,nombreEmpresa]
    return result

def relacionSolvencia(excelFile,nombreEmpresa):
    pass 


def sumatoria(excelFile,listIndex,nombreEmpresa):
    return sum(excelFile.at[i, nombreEmpresa] for i in listIndex)

def indicRelAportesCapital(excelFile,nombreEmpresa): #Indicador de relación entre el Aportes sociales mínimos no reducibles y el Capital Social
    capital_inst_indices = [320000, 330000, 340000]
    capital_inst = sumatoria(excelFile=excelFile,listIndex=capital_inst_indices,nombreEmpresa=nombreEmpresa)
    activo_total = excelFile.at[100000, nombreEmpresa]
    return capital_inst / activo_total if activo_total != 0 else None

#calculo indicadores activos(A)

def indiCalidadPorRiesgo(excelFile,nombreEmpresa):#Indicador de calidad por riesgo
    listCarteraTotal= [ 140410, 140415, 140420, 140425, 140510, 140515, 140520, 140525,
    141110, 141115, 141120, 141125, 141210, 141215, 141220, 141225,
    144110, 144115, 144120, 144125, 144210, 144215, 144220, 144225,
    144810, 144815, 144820, 144825, 145410, 145415, 145420, 145425,
    145510, 145515, 145520, 145525, 146110, 146115, 146120, 146125,
    146210, 146215, 146220, 146225, 146910, 146915, 146920, 146925,
    146935, 146940, 146945, 146950]
    carteraTotal=sumatoria(excelFile=excelFile,listIndex=listCarteraTotal,nombreEmpresa=nombreEmpresa)# Cartera Total Calificada en Categorías B, C, D y E
    
    listCarteraBruta=[ 140400, 140500, 141100, 141200, 144100, 144200,
                        144800, 145400, 145500, 146100, 146200, 146900]
    CarteraBruta=sumatoria(excelFile=excelFile,listIndex=listCarteraBruta,nombreEmpresa=nombreEmpresa) #  Cartera Bruta
    
    return carteraTotal/listCarteraBruta

def CalidadPorRiesgoConCastigo(excelFile,nombreEmpresa): #Indicador de calidad por riesgo con castigos
    carteraCalificadaCategorias=indiCalidadPorRiesgo(excelFile=excelFile,nombreEmpresa=nombreEmpresa)#Cartera Total Calificada en Categorías B, C, D y E + Castigos
    castigos=excelFile.at(831015,nombreEmpresa)
    listTotalCarteraMasCastigos=[ 140400, 140500, 141100, 141200, 144100, 144200, 144800,
                                145400, 145500, 146100, 146200, 146900, 831015]
    totalCarteraMasCastigos=sumatoria(excelFile=excelFile,listIndex=listTotalCarteraMasCastigos,nombreEmpresa=nombreEmpresa)
    return (carteraCalificadaCategorias+castigos)/totalCarteraMasCastigos

def indiCoberturaCarteraTotalRiesgo(excelFile,nombreEmpresa):#Indicador de Cobertura de la Cartera Total en Riesgo
    listDeterioro=[ 140800, 144500, 145100, 145800, 146500, 146800, 147100]
    Deterioro=sumatoria(excelFile=excelFile,listIndex=listDeterioro,nombreEmpresa=nombreEmpresa) #Deterioro
    listCarteraBruta=[ 140400, 140500, 141100, 141200, 144100, 144200,
                        144800, 145400, 145500, 146100, 146200, 146900]
    CarteraBruta=sumatoria(excelFile=excelFile,listIndex=listCarteraBruta,nombreEmpresa=nombreEmpresa) #  Cartera Bruta
    return Deterioro/CarteraBruta

def activoProductivo(excelFile,nombreEmpresa): #Activo Productivo ojo aca
    listactivosProduc=[
    112000, 120000, 130000, 140405, 140410, 140505, 140510,
    141105, 141110, 141205, 141210, 144105, 144110, 144205,
    144210, 144805, 144810, 145405, 145410, 145505, 145510,
    146105, 146110, 146205, 146210, 146905, 146910, 146930,
    146935, 160505, 161505]
    activosProduc=sumatoria(excelFile=excelFile,listIndex=listactivosProduc,nombreEmpresa=nombreEmpresa)
    activo=excelFile.at(100000,nombreEmpresa)
    return listactivosProduc/activo

def IndiCoberturaIndividualCarteraImproductiva(excelFile,nombreEmpresa):# Indicador de Cobertura individual de la cartera improductiva para la cartera en Riesgo 
    ListDeterioroCDE= [
    140815, 140820, 140825,
    144525, 144530, 144535, 144540, 144545, 144550,
    145115, 145120, 145125,
    145825, 145830, 145835, 145840, 145845, 145850,
    146525, 146530, 146535, 146540, 146545, 146550,
    147115, 147120, 147125, 147140, 147145, 147150]
    deterioroCDE=sumatoria(excelFile=excelFile,listIndex=ListDeterioroCDE,nombreEmpresa=nombreEmpresa)
    ListCarteraVencidaCDE=[
    140415, 140420, 140425,140515, 140520, 140525,
    141115, 141120, 141125,141215, 141220, 141225,
    144115, 144120, 144125,144215, 144220, 144225,
    144815, 144820, 144825,145415, 145420, 145425,
    145515, 145520, 145525,146115, 146120, 146125,
    146215, 146220, 146225,146915, 146920, 146925,
    146940, 146945, 146950]
    carteraVencidaCDE=sumatoria(excelFile=excelFile,listIndex=ListCarteraVencidaCDE,nombreEmpresa=nombreEmpresa)
    return deterioroCDE/carteraVencidaCDE

#indicadores de Administracion 


def indiMergenFinancieroOperacion(excelFile,nombreEmpresa): #Indicador de Margen Financiero de Operación
    margenFinancieroOpPositivos=excelFile.at[410000,nombreEmpresa] 
    margenFinancieroOpNegativos=excelFile.at[610000,nombreEmpresa]+excelFile.at[700000,nombreEmpresa]
    ingresosPorVenta=excelFile.at[410000,nombreEmpresa]
    return (margenFinancieroOpPositivos-margenFinancieroOpNegativos)/ingresosPorVenta

def indiMargenOperacional(excelFile,nombreEmpresa):#Indicador de Margen Operacional
    lUtilidadOperacinalP=[410000,422500]
    lUtilidadOperacinalN=[610000,700000,510500,510700,511000,511500,540000]
    lIngresosPorventa=[410000,422500]
    rUtilidadOperacinalP=sumatoria(excelFile=excelFile,listIndex=lUtilidadOperacinalP,nombreEmpresa=nombreEmpresa)
    rUtilidadOperacinalN=sumatoria(excelFile=excelFile,listIndex=lUtilidadOperacinalN,nombreEmpresa=nombreEmpresa)
    rIngresosPorventa=sumatoria(excelFile=excelFile,listIndex=lIngresosPorventa,nombreEmpresa=nombreEmpresa)
    return (rUtilidadOperacinalP-rUtilidadOperacinalN)/rIngresosPorventa

def indiRelOfinancirasPasivos(excelFile,nombreEmpresa): # Indicador de relación entre las obligaciones financieras y el pasivo total
    obligacionFinanciera=excelFile.at[230000,nombreEmpresa]
    pasivoTotal=excelFile.at[200000,nombreEmpresa]
    return obligacionFinanciera/pasivoTotal

def estructuraBalance(ExcelFile,nombreEmpresa):#Estructura de Balance
    lActivoProducido= [
    112000, 120000, 130000,140405, 140410, 140505, 140510,
    141105, 141110, 141205, 141210,144105, 144110, 144205, 
    144210, 144805, 144810,145405, 145410, 145505, 145510,
    146105, 146110, 146205, 146210,146905, 146910, 146930,
    146935,160505, 161505]
    lPasivosCostos=[210000,230000]
    rActivoProducido=sumatoria(excelFile=ExcelFile,listIndex=lActivoProducido,nombreEmpresa=nombreEmpresa)
    rPasivosCostos=sumatoria(excelFile=ExcelFile,listIndex=lPasivosCostos,nombreEmpresa=nombreEmpresa)
    return rActivoProducido/rPasivosCostos

#Rentabilidad





In [18]:
indicRelAportesCapital(data,"COOPANTEX COOPERATIVA DE AHORRO Y CREDITO")

0.039391039042726755

In [5]:
DataFrameModelCamels=readExcelModelCamel(dataModelCamel)
list_cols=["INDICADOR PRINCIPAL","TIPO","Ponderación"]
DataFrameModelCamels=fillCols(execFile=DataFrameModelCamels, cols=list_cols)
DataFrameModelCamels

,INDICADOR PRINCIPAL,TIPO,Ponderación,NOMBRE INDICADOR,Descripcion Indicador,Descripción Formula,Formula
0,C,Capital,0.30,Quebranto Patrimonial,Quebranto Patrimonial,Patrimonio / Capital social,(C300000/C310000)
1,C,Capital,0.30,Relacion Solvencia,Relación de Solvencia,Patrimonio Técnico / Activos Ponderados por Ri...,Se calcula con base en el Anexo 1 del Título I...
2,C,Capital,0.30,Indicador de relación entre el Aportes sociale...,Aportes sociales mínimos no reducibles,Aportes sociales no reducibles / Capital Social,(C311000/C310000)
3,C,Capital,0.30,Indicador de relación entre el Capital Institu...,Capital Institucional,Capital Institucional / Activo,(C32000+C330000+C340000)/C100000
4,A,Activo,0.25,Indicador de calidad por riesgo,"Calidad X Riesgo (B,C,D,E)","Cartera Total Calificada en Categorías B, C, D...",(C140410+C140415+C140420+C140425+C140510+C1405...
5,A,Activo,0.25,Indicador de calidad por riesgo con castigos,"Calidad x Riesgo (B,C,D,E) + Castigos","(Cartera Total Calificada en Categorías B, C, ...",((C140410+C140415+C140420+C140425+C140510+C140...
6,A,Activo,0.25,Indicador de Cobertura de la Cartera Total en ...,Deterioro / Cartera Bruta,Deterioro / Cartera Bruta,(C140800+C144500+C145100+C145800+C146500+C1468...
7,A,Activo,0.25,Activo Productivo,Activo Productivo / Activo,Activos Productivos / Activo,(C112000+C120000+C130000+C140405+C140410+C1405...
8,A,Activo,0.25,Indicador de Cobertura individual de la carter...,Cobertura C D y E,Deterioro CDE / Cartera Vencida CDE,(C140815+C140820+C140825+C144525+C144530+C1445...
9,M,Administración,0.10,Indicador de Margen Financiero de Operación,Margen Financiero de Operación,Margen Financiero de Operación / Ingresos por ...,((C410000-C610000-C700000)/C410000)
